In [8]:
import os
import cv2
import numpy as np

class ImprovedMannequinRemover:
    def __init__(self):
        print("Model loaded successfully!")

    def ensure_directory_exists(self, folder_path):
        """Ensure that the directory exists, create if not."""
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    def resize_image(self, image, target_size=512):
        """Resize image maintaining aspect ratio"""
        h, w = image.shape[:2]
        aspect = w / h

        if h > w:
            new_h = target_size
            new_w = int(target_size * aspect)
        else:
            new_w = target_size
            new_h = int(target_size / aspect)

        return cv2.resize(image, (new_w, new_h))

    def pad_image(self, image, target_size=512):
        h, w = image.shape[:2]
        top = (target_size - h) // 2
        bottom = target_size - h - top
        left = (target_size - w) // 2
        right = target_size - w - left

        return cv2.copyMakeBorder(image, top, bottom, left, right,
                                  cv2.BORDER_CONSTANT, value=[255, 255, 255])

    def create_refined_mask(self, image):
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        lower_white1 = np.array([0, 0, 200])
        upper_white1 = np.array([180, 30, 255])
        mask1 = cv2.inRange(hsv, lower_white1, upper_white1)

        lower_white2 = np.array([0, 0, 130])
        upper_white2 = np.array([180, 40, 255])
        mask2 = cv2.inRange(hsv, lower_white2, upper_white2)

        combined_mask = cv2.bitwise_or(mask1, mask2)

        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        return mask

    def process_image(self, input_path, resized_folder, masks_folder):
        try:
            print(f"Processing image: {input_path}")
            original_image = cv2.imread(input_path)
            if original_image is None:
                raise ValueError(f"Could not read image at {input_path}")

            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

            print("Preprocessing image...")
            resized_image = self.resize_image(original_image)
            padded_image = self.pad_image(resized_image)

            # Ensure output directories exist
            self.ensure_directory_exists(resized_folder)
            self.ensure_directory_exists(masks_folder)

            # Save resized image
            image_filename = os.path.basename(input_path)
            resized_image_path = os.path.join(resized_folder, f"resized_{image_filename}")
            padded_image = cv2.cvtColor(padded_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(resized_image_path, padded_image)

            print(f"Resized image saved to: {resized_image_path}")

            print("Creating mask...")
            mask = self.create_refined_mask(padded_image)

            # Save mask
            mask_image_path = os.path.join(masks_folder, f"mask_{image_filename}")
            cv2.imwrite(mask_image_path, mask)

            print(f"Mask saved to: {mask_image_path}")

            return resized_image_path, mask_image_path

        except Exception as e:
            print(f"Error: {str(e)}")
            raise

def expand_mask(mask, expansion_pixels=10):
    kernel = np.ones((expansion_pixels - 60, expansion_pixels - 60), np.uint8)
    expanded_mask = cv2.dilate(mask, kernel, iterations=1)
    return expanded_mask

def apply_inverse_mask(image_path, mask_path, output_folder):
    print(f"Applying inverse mask on: {image_path}")

    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    _, mask = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY)

    if image.shape[:2] != mask.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    mask_inv = cv2.bitwise_not(mask)
    expanded_mask = expand_mask(mask_inv, 60)

    white_background = np.ones_like(image) * 255
    result = np.where(expanded_mask[:, :, None] == 255, image, white_background)

    final_output_path = os.path.join(output_folder, f"output_{os.path.basename(image_path)}")
    cv2.imwrite(final_output_path, result)

    print(f"Processed image saved to: {final_output_path}")

def process_folder(input_folder, resized_folder, masks_folder, output_folder):

    if not os.path.exists(input_folder):
        raise ValueError(f"Input folder does not exist: {input_folder}")

    print(f"Ensuring output directories are set up.")
    if not os.path.exists(resized_folder):
        os.makedirs(resized_folder)
    if not os.path.exists(masks_folder):
        os.makedirs(masks_folder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    remover = ImprovedMannequinRemover()

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)

            # Process image and create mask
            resized_image_path, mask_image_path = remover.process_image(input_path, resized_folder, masks_folder)

            # Apply inverse mask
            apply_inverse_mask(resized_image_path, mask_image_path, output_folder)

def main():
    # Define folder paths
    input_folder = "E:\wetransfer_pgotoes_2024-09-06_1656\27_02_24\P1_27_02\1"        # Replace with your input folder path
    resized_folder = "/content/resized_images"
    masks_folder = "/content/masks"
    output_folder = "/content/output_images"

    process_folder(input_folder, resized_folder, masks_folder, output_folder)


if __name__ == "__main__":
    main()


Ensuring output directories are set up.
Model loaded successfully!
Processing image: /content/testing/1170457371073_1.jpg
Preprocessing image...
Resized image saved to: /content/resized_images/resized_1170457371073_1.jpg
Creating mask...
Mask saved to: /content/masks/mask_1170457371073_1.jpg
Applying inverse mask on: /content/resized_images/resized_1170457371073_1.jpg
Processed image saved to: /content/output_images/output_resized_1170457371073_1.jpg
Processing image: /content/testing/1170457351259_1.jpg
Preprocessing image...
Resized image saved to: /content/resized_images/resized_1170457351259_1.jpg
Creating mask...
Mask saved to: /content/masks/mask_1170457351259_1.jpg
Applying inverse mask on: /content/resized_images/resized_1170457351259_1.jpg
Processed image saved to: /content/output_images/output_resized_1170457351259_1.jpg
